## MAX-MIN CONVOLUTIONAL NEURAL NETWORKS FOR IMAGE CLASSIFICATION

You can find the original paper here  [Max-Min CNN](http://webia.lip6.fr/~thomen/papers/Blot_ICIP_2016.pdf)

Also you can look at the comparision of performance here [Classical CNN vs MaxMin CNN](https://github.com/kdexd/maxmin-cnn/blob/master/cifar10_classical_vs_maxmin_baseline.ipynb)
## Paper Notes
* In this paper, we propose to modify the standard convolutional
block of CNN in order to **transfer more information
layer after layer while keeping some invariance within the network**.
* Our main idea is to **exploit both positive and negative
high scores** obtained in the convolution maps.
* We are **doubling the maps** with specific
activations functions, called MaxMin strategy, in order to
achieve our pipeline.
* In classical CNN , using a ReLU activation function,
all negative information is removed from the convolutional
map considered. 
* To implement our strategy :
> * We **duplicate the convolutional filter maps** (represented is blue in Fig. 1) and **multiply the
copy by × − 1** resulting in the negative version of the detections
(in red in Fig. 1). 
> * We then **concatenate the original
maps and their negative copy** as shown on Fig. 1. This operation
increases the depth of the convolutional layer’s output
by two compared to classical CNN. 
> * We then **apply the ReLU normally to the concatenated output** and optionally process a
pooling operation. This way negative values are not filtered
and can be exploited by following layers.
![MaxMinScheme](https://photos-4.dropbox.com/t/2/AABdvwxmw0UZCeLE4ALrZCsmadYp1TyKSvk3to2vR-FGSQ/12/478136838/png/32x32/1/_/1/2/MaxMinScheme.PNG/EOnm8-8DGAsgBygH/KKsDP1vBAO-J1-nZh58gx7CNL7EuZVZO5QYsHdXCAE0?preserve_transparency=1&size=1600x1200&size_mode=3)

> Look at the following line in the code for [MaxMin Convolution class](https://github.com/kdexd/maxmin-cnn/blob/master/keras_maxmin_impl.py), which performs the above operations.
```
output = K.concatenate([output, -output], axis=3)
```

![MaxMin CNN Block](https://photos-2.dropbox.com/t/2/AABCnfZGeS_cdGDOQvUiyrrITdO6vZ9NRRj9ZP1b0BPgxw/12/478136838/png/32x32/1/_/1/2/MaxMinScheme1.PNG/EOnm8-8DGAwgBygH/yzWQgBbYhMvegSo5Ha-K1YcygTCiHRnSXNZf8gy1pgo?preserve_transparency=1&size=1600x1200&size_mode=3)

*  Fig. 2 shows that
different versions of a same shape (yellow and blue triangles)
can be transmitted with MaxMin network with only 1 filter,
whereas 2 filters are necessary for standard CNN. Thus **we
can reduce the number of filters and keep a reasonable amount
of parameters.**

* MaxMin networks learn each pattern both from its
positive and negative occurrences in the dataset. They learn
the pattern filters more accurately and faster than a classical
CNN that would learn positive and negative filters independently.

* **Generalization**: On a spatial window, the maximum and
the minimum pooling outputs are rarely simultaneously positively
high and negatively low. Therefore, ReLU filters very
often one of the two values (min or max). This ensures a
sparse activation of the network’s neurons. This property is
known to enhance the generalization performance of neural
networks in computer vision.

* ** Why this architecture is called MaxMin CNN ?**
> * After the ReLU is frequently applied max pooling map by
map. 
> * Also note that it is possible to commute the two layers without changing the
block’s output. ie.  
```
ReLU o  Max = Max o ReLU
```
> * For our method we notice the fact that
>>   **( Max(ReLU(X)) , Max(ReLU( - X)) ) = ( ReLU(Max(X)) , ReLU( - Min(X)) )**
>  where X is a vector. 

> Thus our method can be interpreted as
simply adding an additional information at pooling function
with a bi-dimensional output when applied before the ReLU.
This additional information is the minimum detection on the
window taken negatively.



## Keras implementation of MaxMinConvolution2D
This is a sample implementation of the MaxMin Convolution described above in Keras with tensorflow backend.

In [1]:
from keras import backend as K
from keras.layers.convolutional import _Conv
from keras import activations
from keras.engine import InputSpec
from keras.layers.convolutional import Conv2D

class MaxMinConvolution2D(Conv2D):

    def __init__(self, filters,
                 kernel_size,
                 strides=(1, 1),
                 padding='valid',
                 data_format=None,
                 dilation_rate=(1, 1),
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(MaxMinConvolution2D, self).__init__(

            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            **kwargs)

    def build(self, input_shape):
        super(MaxMinConvolution2D, self).build(input_shape)

    def call(self, x):

        output = super(MaxMinConvolution2D, self).call(x)
        output = K.concatenate([output, -output], axis=3)
        return output

    def compute_output_shape(self, input_shape):
        """The output shape is doubled along the axis representing channels due
           to concatenation of two identical sized Convolution layers.
        """
        output_shape = super(MaxMinConvolution2D, self).compute_output_shape(input_shape)

        output_shape = list(output_shape)

        output_shape[3] *= 2

        return tuple(output_shape)

Using TensorFlow backend.


## Code for Image Classification on CIFAR-10 Dataset
In this section we'll use the MaxMin CNN concept and apply it on CIFAR-10 dataset image classification.

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 256
num_classes = 10
epochs = 15
l = 40
num_filter = 32
compression = 0.8
dropout_rate = 0.25

In [6]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 27s 0us/step


In [0]:
from keras.layers import LeakyReLU

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = LeakyReLU(alpha=0.1)(BatchNorm)
        Conv2D_3_1 = MaxMinConvolution2D(int(num_filter*compression), (5,1), use_bias=False ,padding='same')(relu)
        Conv2D_3_3 = MaxMinConvolution2D(int(num_filter*compression), (1,5), use_bias=False ,padding='same')(Conv2D_3_1)

        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = LeakyReLU(alpha=0.1)(BatchNorm)
    Conv2D_BottleNeck = MaxMinConvolution2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = LeakyReLU(alpha=0.1)(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 32
dropout_rate = 0.25
l = 12
input = Input(shape=(img_height, img_width, channel))

In [0]:
First_Conv2D_3_1 = MaxMinConvolution2D(num_filter, (7,1), use_bias=False ,padding='same')(input)
First_Conv2D = MaxMinConvolution2D(num_filter, (1,7), use_bias=False ,padding='same')(First_Conv2D_3_1)

In [13]:
First_Conv2D

<tf.Tensor 'max_min_convolution2d_2/concat:0' shape=(?, 32, 32, 64) dtype=float32>

In [0]:
First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [15]:
output

<tf.Tensor 'dense_1/Softmax:0' shape=(?, 10) dtype=float32>

In [0]:
num_filter = 16
dropout_rate = 0.5
l = 12
input = Input(shape=(img_height, img_width, channel,))

First_Conv2D_3_1 = MaxMinConvolution2D(num_filter, (7,1), use_bias=False ,padding='same')(input)
First_Conv2D = MaxMinConvolution2D(num_filter, (1,7), use_bias=False ,padding='same')(First_Conv2D_3_1)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [17]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
max_min_convolution2d_102 (MaxM (None, 32, 32, 32)   336         input_2[0][0]                    
__________________________________________________________________________________________________
max_min_convolution2d_103 (MaxM (None, 32, 32, 32)   3584        max_min_convolution2d_102[0][0]  
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 32)   128         max_min_convolution2d_103[0][0]  
__________________________________________________________________________________________________
leaky_re_l


batch_normalization_101 (BatchN (None, 4, 4, 240)    960         concatenate_93[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_101 (LeakyReLU)     (None, 4, 4, 240)    0           batch_normalization_101[0][0]    
__________________________________________________________________________________________________
max_min_convolution2d_197 (MaxM (None, 4, 4, 24)     14400       leaky_re_lu_101[0][0]            
__________________________________________________________________________________________________
max_min_convolution2d_198 (MaxM (None, 4, 4, 24)     1440        max_min_convolution2d_197[0][0]  
__________________________________________________________________________________________________
dropout_100 (Dropout)           (None, 4, 4, 24)     0           max_min_convolution2d_198[0][0]  
__________________________________________________________________________________________________
concatena

In [0]:
# determine Loss function and Optimizer
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )
datagen.fit(x_train)

In [0]:
model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
43776/50000 [=========================>....] - ETA: 58s - loss: 1.7634 - acc: 0.3355

50000/50000 [==============================] - 499s 10ms/step - loss: 1.7310 - acc: 0.3486 - val_loss: 1.7815 - val_acc: 0.4298
Epoch 2/50
26112/50000 [==============>...............] - ETA: 3:35 - loss: 1.3849 - acc: 0.4922

50000/50000 [==============================] - 481s 10ms/step - loss: 1.3355 - acc: 0.5122 - val_loss: 1.4679 - val_acc: 0.5073
Epoch 3/50
19392/50000 [==========>...................] - ETA: 4:36 - loss: 1.1746 - acc: 0.5750

50000/50000 [==============================] - 483s 10ms/step - loss: 1.1315 - acc: 0.5920 - val_loss: 1.1777 - val_acc: 0.5969
Epoch 4/50
16896/50000 [=========>....................] - ETA: 4:59 - loss: 1.0407 - acc: 0.6257

50000/50000 [==============================] - 484s 10ms/step - loss: 1.0074 - acc: 0.6378 - val_loss: 0.9898 - val_acc: 0.6536
Epoch 5/50
15936/50000 [========>.....................] - ETA: 5:07 - loss: 0.9248 - acc: 0.6697

...

50000/50000 [==============================] - 481s 10ms/step - loss: 0.4087 - acc: 0.8582 - val_loss: 1.1432 - val_acc: 0.7026
Epoch 33/50
15232/50000 [========>.....................] - ETA: 5:12 - loss: 0.4115 - acc: 0.8549

50000/50000 [==============================] - 481s 10ms/step - loss: 0.4069 - acc: 0.8580 - val_loss: 1.2792 - val_acc: 0.6677
Epoch 34/50
15232/50000 [========>.....................] - ETA: 5:12 - loss: 0.3983 - acc: 0.8631

50000/50000 [==============================] - 481s 10ms/step - loss: 0.4023 - acc: 0.8594 - val_loss: 1.6960 - val_acc: 0.6146
Epoch 35/50
15232/50000 [========>.....................] - ETA: 5:12 - loss: 0.3780 - acc: 0.8681

50000/50000 [==============================] - 481s 10ms/step - loss: 0.3916 - acc: 0.8633 - val_loss: 0.8469 - val_acc: 0.7571
Epoch 36/50
15232/50000 [========>.....................] - ETA: 5:13 - loss: 0.3890 - acc: 0.8664

50000/50000 [==============================] - 479s 10ms/step - loss: 0.3895 - acc: 0.8656 - val_loss: 1.4754 - val_acc: 0.6373
Epoch 37/50
 5440/50000 [==>...........................] - ETA: 6:35 - loss: 0.3746 - acc: 0.8732

In [41]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 31s 3ms/step
Test loss: 0.808814356827736
Test accuracy: 0.7619


In [42]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')

## References
1.   Max Min Convolutional Neural Networks for Image Classification by Blot et. al (2016).

